<a href="https://colab.research.google.com/github/gaurxvreddy/ProjectBlind/blob/main/ProjectNotBlind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

In [13]:
!git clone --depth 1 https://github.com/tensorflow/models

fatal: destination path 'models' already exists and is not an empty directory.


In [14]:
!sudo apt install -y protobuf-compiler
%cd models/research/
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
/content/models/research
Processing /content/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1675218 sha256=580070677451163f84515f8a9c70cc710addfa1bbe1bf65055314bc09adf4076
  Stored in directory: /tmp/pip-ephem-wheel-cache-rltjobnx/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
Successfully built 

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

In [17]:
%cd /content/
!curl -L "https://app.roboflow.com/ds/CrESyj8JxX?key=QheQbcXOwu" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

/content
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   891  100   891    0     0   1400      0 --:--:-- --:--:-- --:--:--  1400
100  203M  100  203M    0     0  83.5M      0  0:00:02  0:00:02 --:--:--  132M
Archive:  roboflow.zip
replace README.roboflow.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train/person.tfrecord? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train/person_label_map.pbtxt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace valid/person.tfrecord? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace valid/person_label_map.pbtxt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [18]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
!tar -xvf "/content/efficientdet_d0_coco17_tpu-32.tar.gz"
!rm "/content/efficientdet_d0_coco17_tpu-32.tar.gz"
!mkdir "/content/model/"
!cp "/content/efficientdet_d0_coco17_tpu-32/pipeline.config" "/content/model/"

--2021-10-18 18:56:34--  http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.217.128, 2607:f8b0:400c:c13::80
Connecting to download.tensorflow.org (download.tensorflow.org)|173.194.217.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30736482 (29M) [application/x-tar]
Saving to: ‘efficientdet_d0_coco17_tpu-32.tar.gz’

efficientdet_d0_coc 100%[===================>]  29.31M   106MB/s    in 0.3s    

2021-10-18 18:56:35 (106 MB/s) - ‘efficientdet_d0_coco17_tpu-32.tar.gz’ saved [30736482/30736482]

efficientdet_d0_coco17_tpu-32/
efficientdet_d0_coco17_tpu-32/checkpoint/
efficientdet_d0_coco17_tpu-32/checkpoint/ckpt-0.data-00000-of-00001
efficientdet_d0_coco17_tpu-32/checkpoint/checkpoint
efficientdet_d0_coco17_tpu-32/checkpoint/ckpt-0.index
efficientdet_d0_coco17_tpu-32/pipeline.config
efficientdet_d0_coco17_tpu-32/saved_model/
efficientdet

In [19]:
!python /content/models/research/object_detection/model_main_tf2.py --model_dir=/content/model --pipeline_config_path=/content/model/pipeline.config

2021-10-18 19:02:06.860198: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-18 19:02:06.860289: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ae8984f642f6): /proc/driver/nvidia/version does not exist
W1018 19:02:06.925302 140003697878912 cross_device_ops.py:1387] There are non-GPU devices in `tf.distribute.Strategy`, not using nccl allreduce.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
I1018 19:02:07.074313 140003697878912 mirrored_strategy.py:369] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: None
I1018 19:02:07.112138 140003697878912 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1018 19:02:07.112877 140003697878912 

In [ ]:
model = hub.load("https://tfhub.dev/tensorflow/efficientdet/d0/1")

In [ ]:
img = cv2.imread("/content/output_hX-AWUQ1wIEr_0.png")

In [ ]:
results = model(tf.expand_dims(img, axis=0))

In [ ]:
class_ids = results["detection_classes"]
class_ids

<tf.Tensor: shape=(1, 100), dtype=float32, numpy=
array([[38.,  1.,  1., 38.,  1.,  1.,  1.,  1., 38., 38.,  1.,  1., 38.,
         1.,  1., 38.,  1.,  1.,  1., 38.,  1.,  1., 38.,  1., 16.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., 38.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., 16.,  1.,
         1., 16., 42., 38.,  1.,  1.,  1., 38., 16.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1., 42., 16.,  1., 38.,  1.,  1.,  1., 42.,
         1.,  1.,  9., 38., 38., 42., 38., 38.,  1., 16.,  1.,  1.,  1.,
        38., 38.,  1.,  1.,  1.,  1., 16., 42., 16.]], dtype=float32)>

In [ ]:
result = {key:value.numpy() for key,value in results.items()}
print(result.keys())

dict_keys(['raw_detection_scores', 'detection_multiclass_scores', 'detection_scores', 'raw_detection_boxes', 'detection_anchor_indices', 'num_detections', 'detection_classes', 'detection_boxes'])


In [ ]:
result

{'detection_anchor_indices': array([[ 3150., 20851., 18511.,  7032., 13911., 12903., 13875., 13272.,
          9453.,  6456., 12912., 12255.,  9462., 12246., 13281.,  9408.,
         11388., 12894., 11397.,  5880., 11058., 13920.,  9417., 12264.,
         13620., 10812., 10257., 11049., 13884., 13290., 10821., 12822.,
         13488., 11085., 11670., 11076., 12018.,  8322., 13827., 11802.,
         11067., 11094., 11031., 12594., 12318., 11040., 13902., 13335.,
         13893., 13983., 13611., 11688., 13620., 14187., 13911.,  8886.,
         12378., 22578., 11679.,  8877., 14196., 11013.,  7032., 12327.,
         11112., 11724.,  9453., 13836., 11103., 12291., 11397., 11388.,
         12723.,  9390., 11190., 12300., 12921., 11388., 11415., 12696.,
         11388., 11085., 11190., 12903.,  9357., 10029., 12936., 15321.,
         11238., 11175., 11184., 11094.,  9444., 14559., 11715., 11217.,
         10677., 12018., 13620., 14151.]], dtype=float32),
 'detection_boxes': array([[[0.095516

In [ ]:
PATH_TO_LABELS = '/content/models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
# Handle models with masks:
image_np_with_mask = np.reshape(img, (1, 926, 1379, 3))

if 'detection_masks' in result:
  # we need to convert np.arrays to tensors
  detection_masks = tf.convert_to_tensor(result['detection_masks'][0])
  detection_boxes = tf.convert_to_tensor(result['detection_boxes'][0])

  # Reframe the the bbox mask to the image size.
  detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes,
              image_np.shape[1], image_np.shape[2])
  detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                      tf.uint8)
  result['detection_masks_reframed'] = detection_masks_reframed.numpy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_mask[0],
      result['detection_boxes'][0],
      (result['detection_classes'][0] + label_id_offset).astype(int),
      result['detection_scores'][0],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.30,
      agnostic_mode=False,
      instance_masks=result.get('detection_masks_reframed', None),
      line_thickness=8)

plt.figure(figsize=(24,32))
plt.imshow(image_np_with_mask[0])
plt.show()

NameError: ignored

In [ ]:
!curl -L "https://public.roboflow.com/ds/QW7s6FWFz2?key=338vuOF3bs" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   883  100   883    0     0   1431      0 --:--:-- --:--:-- --:--:--  1428
100 38.2M  100 38.2M    0     0  34.7M      0  0:00:01  0:00:01 --:--:-- 34.7M
Archive:  roboflow.zip
replace README.dataset.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 extracting: README.dataset.txt      
replace README.roboflow.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 extracting: README.roboflow.txt     
replace test/vehicles.tfrecord? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 extracting: test/vehicles.tfrecord  
replace test/vehicles_label_map.pbtxt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 extracting: test/vehicles_label_map.pbtxt  
replace train/vehicles.tfrecord? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 extracting: train/vehicles.tfrecord  
replace train/vehicles_label_map.pbtxt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 extracting: train/veh

In [ ]:
y%ls